In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import itertools
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import pprint


In [ ]:
tf.__version__

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
heart_failure_clinical_records = pd.read_csv(os.path.join(dirname, filename))

In [ ]:
def get_training_and_test_ds(raw_ds, train_ratio=0.8):
    raw_features = raw_ds.copy()
    raw_labels = raw_features.pop('DEATH_EVENT')
    return train_test_split(raw_features, raw_labels,test_size=1-train_ratio,train_size=train_ratio)
# features_train, features_test,label_train, label_test

In [ ]:
def get_train_and_test_ds_ver2():
    new_raw_ds = heart_failure_clinical_records.copy()
    train_ds = new_raw_ds.sample(frac=0.8, random_state=0)
    test_ds = new_raw_ds.drop(train_ds.index)
    return train_ds, test_ds

In [ ]:
features_train, features_test,label_train, label_test = get_training_and_test_ds(heart_failure_clinical_records)
features_train.info()

In [ ]:
sns.pairplot(heart_failure_clinical_records[['age', 'DEATH_EVENT']], diag_kind='kde')

In [ ]:
# print(features_test.shape())
# print(label_train.count())
# print(label_test.count())
tf.keras.backend.clear_session()

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation = 'relu', 
                          activity_regularizer=tf.keras.regularizers.L2(0.01)),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
%%time
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("heart_failure_model.h5")
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10)
history = model.fit(features_train, 
                    label_train, 
                    epochs=100, 
                    verbose=0,
                    validation_split=0.2,
                    callbacks=[model_checkpoint, early_stopping])

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()